In [1]:
import numpy as np
import math

import sys
sys.path.insert(0, "../")

from denseCNN import denseCNN

In [2]:
arrange8x8 = np.array([
    28,29,30,31,0,4,8,12,
    24,25,26,27,1,5,9,13,
    20,21,22,23,2,6,10,14,
    16,17,18,19,3,7,11,15,
    47,43,39,35,35,34,33,32,
    46,42,38,34,39,38,37,36,
    45,41,37,33,43,42,41,40,
    44,40,36,32,47,46,45,44])
arrMask  =  np.array([
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,           
    1,1,1,1,1,1,1,1,           
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,1,0,
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,1,0,0,]) 
calQMask  =  np.array([
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,           
    1,1,1,1,1,1,1,1,           
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,0,0,]) 

f_raw = '../SampleSplitting_SignalAllocation/nElinks_5/ttbar_v11_eolNoise_Layer9_5Links.csv'
calQ     = np.genfromtxt(f_raw, delimiter=',',usecols=[*range(0, 48)],skip_header=1,max_rows=100)
sumQ     = calQ.sum(axis=1)
calQ     = calQ[sumQ>0]
sumQ     = sumQ[sumQ>0]

inputQf_48_arr   = np.array([calQ[i]/s for i,s in enumerate(sumQ)])[:,arrange8x8]  
inputQf_48_arr_mask   = np.array([calQ[i]/s for i,s in enumerate(sumQ)])[:,arrange8x8]  
inputQf_48_arr_mask[:,arrMask==0]=0

In [3]:
print("Before mask")
print(np.round(inputQf_48_arr[1].reshape(8,8),3))
print("After mask")
print(np.round(inputQf_48_arr_mask[1].reshape(8,8),3))


Before mask
[[0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.286 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.286 0.    0.   ]
 [0.    0.214 0.    0.286 0.    0.    0.    0.   ]
 [0.214 0.    0.    0.    0.    0.214 0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.214 0.   ]]
After mask
[[0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.286 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.214 0.    0.286 0.    0.    0.    0.   ]
 [0.214 0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]]


In [54]:
m = denseCNN()
m.setpams({
    'arrMask':arrMask,'arrange':arrange8x8,
     'shape':(8,8,1),'channels_first':False,'encoded_dim':16})
m.init()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 8, 8, 1)]         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 8)           80        
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
encoded_vector (Dense)       (None, 16)                8208      
Total params: 8,288
Trainable params: 8,288
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 16)]              0         
________________________________________

In [67]:
x = m.predict(np.arange(64).reshape(1,8,8,1))

In [68]:
x[0].reshape(8,8)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])

In [69]:
np.unique(m.mapToCalQ(x[0]).reshape(3,4,4))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 41, 42, 43, 48, 49, 50, 51, 56, 57, 58, 59, 61])

In [75]:
print(arrange8x8)
m.invertArrange(arrange8x8,arrMask)

[28 29 30 31  0  4  8 12 24 25 26 27  1  5  9 13 20 21 22 23  2  6 10 14
 16 17 18 19  3  7 11 15 47 43 39 35 35 34 33 32 46 42 38 34 39 38 37 36
 45 41 37 33 43 42 41 40 44 40 36 32 47 46 45 44]


array([ 4, 12, 20, 28,  5, 13, 21, 29,  6, 14, 22, 30,  7, 15, 23, 31, 24,
       25, 26, 27, 16, 17, 18, 19,  8,  9, 10, 11,  0,  1,  2,  3, 59, 51,
       43, 35, 58, 50, 42, 34, 57, 49, 41, 33, 56, 48, 61, 32])

In [8]:
def invertArrange(arrange,arrMask=[],calQMask=[]):
    remap =[]
    hashmap = {}  ## cell:index mapping
    ##Valid arrange check
    if not np.all(np.unique(arrange)==np.arange(48)):
        raise ValueError("Found cell location with number > 48. Please check your arrange:",arrange)
    foundDuplicateCharge = False
    if len(arrMask)==0:
        if len(arrange)>len(np.unique(arrange)):
            foundDuplicateCharge=True
    else:
        if len(arrange[arrMask==1])>len(np.unique(arrange[arrMask==1])):
            foundDuplicateCharge=True

    if foundDuplicateCharge and len(calQMask)==0:
        raise ValueError("Found duplicated charge arrangement, but did not specify calQmask")  
        
    for i in range(len(arrange)):
        if len(arrMask)>0 :
            ## fill hashmap only if arrMask allows it
            if arrMask[i]==1:   
                if(foundDuplicateCharge and calQMask[i]==1):
                    ## fill hashmap only if calQMask allows it
                    hashmap[arrange[i]]=i                    
        else:
            hashmap[arrange[i]]=i
    for i in range(len(np.unique(arrange))):
        remap.append(hashmap[i])
    return np.array(remap)

In [9]:
arrange8x8 = np.array([
    28,29,30,31,0,4,8,12,
    24,25,26,27,1,5,9,13,
    20,21,22,23,2,6,10,14,
    16,17,18,19,3,7,11,15,
    47,43,39,35,35,34,33,32,
    46,42,38,34,39,38,37,36,
    45,41,37,33,43,42,41,40,
    44,40,36,32,47,46,45,44])
arrMask  =  np.array([
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,           
    1,1,1,1,1,1,1,1,           
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,1,0,
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,1,0,0,]) 
calQMask  =  np.array([
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,           
    1,1,1,1,1,1,1,1,           
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,0,0,
    1,1,1,1,0,0,0,0,]) 
remap = invertArrange(arrange8x8,arrMask,calQMask)


In [189]:
print('before map')
print(np.arange(48).reshape(6,8))
print('After map (physical localtion)')
print(np.arange(48)[arrange8x8].reshape(8,8))
print('After mask')
x = np.arange(64)[arrange8x8]
x[arrMask==0]=0
print(x.reshape(8,8))

print('After remap')
print(len(np.arange(64)[remap]))
print(x[remap].reshape(6,8))

before map
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]]
After map (physical localtion)
[[28 29 30 31  0  4  8 12]
 [24 25 26 27  1  5  9 13]
 [20 21 22 23  2  6 10 14]
 [16 17 18 19  3  7 11 15]
 [47 43 39 35 35 34 33 32]
 [46 42 38 34 39 38 37 36]
 [45 41 37 33 43 42 41 40]
 [44 40 36 32 47 46 45 44]]
After mask
[[28 29 30 31  0  4  8 12]
 [24 25 26 27  1  5  9 13]
 [20 21 22 23  2  6 10 14]
 [16 17 18 19  3  7 11 15]
 [47 43 39 35  0  0  0  0]
 [46 42 38 34  0  0 37  0]
 [45 41 37 33  0  0  0  0]
 [44 40 36 32  0 46  0  0]]
After remap
48
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]]


In [7]:
np.count_nonzero(calQMask)==48

True